In [1]:
import pyomo.environ as pyo
import json
from datetime import timedelta
import polars as pl
from polars  import col as c
import os 
import numpy as np
import math
import tqdm
from datetime import timedelta, datetime, timezone
from optimization_model.optimizaztion_pipeline import first_stage_pipeline
from optimization_model.input_data_prepocessing import generate_first_problem_input_data
from data_display.first_stage_optimization_plots import plot_first_stage_summarized
from data_display.input_data_plots import plot_basin_height_volume_table
from utility.pyomo_preprocessing import extract_optimization_results
from utility.polars_operation import linear_interpolation_for_bound, arange_float
from data_federation.input_model import SmallflexInputSchema
from utility.pyomo_preprocessing import generate_datetime_index, generate_clean_timeseries, optimal_segments, generate_segments, process_performance_table
from config import settings
from utility.general_function import pl_to_dict, build_non_existing_dirs
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
COLORS = px.colors.qualitative.Plotly


from optimization_model.first_model_baseline import generate_baseline_model

In [2]:
os.chdir(os.getcwd().replace("/src", ""))
os.environ['GRB_LICENSE_FILE'] = '/home/ltomasini/gruobi_license/gurobi.lic'


In [3]:
output_file_names: dict[str, str] = json.load(open(settings.OUTPUT_FILE_NAMES))

small_flex_input_schema: SmallflexInputSchema = SmallflexInputSchema()\
.duckdb_to_schema(file_path=output_file_names["duckdb_input"])

Read and validate tables from small_flex_input_data.db file: 100%|████████████████████████████████████████████████████| 12/12 [00:01<00:00,  6.89it/s]


In [10]:
folder_name = ".cache/plot"
build_non_existing_dirs(folder_name)
dict_plot = plot_basin_height_volume_table(small_flex_input_schema=small_flex_input_schema)
fig = dict_plot["Aegina upstream basin"]

fig.update_layout(
    title="Greis lac height-volume curve",
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
    yaxis=dict(
        nticks=5  # Approximate number of ticks on the y-axis
    ),
    font=dict(
        family="Georgia",
        size=18,
        color="Black"
    )
)
fig.write_image(folder_name + "/griessee_height_volume.png")
fig.show()

In [5]:
market_country = "CH"
market = "DA"
n_segments = 5

year = 2020
max_flow_factor: float = 1
min_flow_factor: float = 0

solver = pyo.SolverFactory('gurobi')
solver.options['MIPGap'] = 2e-3
for with_pumping, add_wind_production in [(False, False), (True, False), (True, True)]:
    turbined_volume = {}
    pumped_volume = {}
    model: pyo.AbstractModel = generate_baseline_model(with_pumping=with_pumping)
    nb_days_list = [7]
    for nb_days in nb_days_list:
        
        data, turbined_table_per_volume = generate_first_problem_input_data(
            small_flex_input_schema=small_flex_input_schema, 
            hydro_power_plant_name= "Aegina hydro",
            n_segments=n_segments, 
            year=year, 
            max_flow_factor=max_flow_factor,
            min_flow_factor=min_flow_factor, 
            first_time_delta=timedelta(days=nb_days),
            add_wind_production=add_wind_production
        )
        model_instance: pyo.Model = model.create_instance({None: data})
        with tqdm.tqdm(total=1, desc="Solving baseline first stage optimization") as pbar:
            _ = solver.solve(model_instance, load_solutions=True, tee=False)
            pbar.update(1)  
    
        plot_first_stage_summarized(
            with_pumping=with_pumping,
            model_instance=model_instance, 
            min_flow_factor=min_flow_factor, 
            max_flow_factor=max_flow_factor, 
            nb_days=nb_days, 
            year=year)
        
        factor: int = nb_days_list[0]//nb_days
        
        turbined_volume[nb_days] = extract_optimization_results(model_instance, "turbined_volume")\
            .with_columns(c("T")//factor).group_by("T").agg(c("turbined_volume").sum()).sort("T")
        if with_pumping:
            pumped_volume[nb_days] = extract_optimization_results(model_instance, "pumped_volume")\
                .with_columns(c("T")//factor).group_by("T").agg(c("pumped_volume").sum()).sort("T")

Solving baseline first stage optimization: 100%|██████████| 1/1 [00:01<00:00,  1.93s/it]

{'Income': 0.9116625330277445, 'turbined_volume': 23.820974099999983, 'discharge_volume': 23.8209741}


Solving baseline first stage optimization: 100%|██████████| 1/1 [00:12<00:00, 12.05s/it]

{'Income': 0.9894584906062979, 'turbined_volume': 35.382095209999996, 'discharge_volume': 23.8209741, 'pumped_volume': 11.561121109999998}


Solving baseline first stage optimization: 100%|██████████| 1/1 [00:10<00:00, 10.31s/it]

{'Income': 1.2174816308084162, 'turbined_volume': 35.382095209999996, 'discharge_volume': 23.8209741, 'pumped_volume': 11.561121109999998}


In [6]:
extract_optimization_results(model_instance, "alpha_turbined")

shape: (5, 2)
┌─────┬────────────────┐
│ H   ┆ alpha_turbined │
│ --- ┆ ---            │
│ i64 ┆ f64            │
╞═════╪════════════════╡
│ 0   ┆ 3.150375       │
│ 1   ┆ 3.2072         │
│ 2   ┆ 3.285782       │
│ 3   ┆ 3.388958       │
│ 4   ┆ 3.457244       │
└─────┴────────────────┘

In [7]:
wind_production = small_flex_input_schema.power_production_measurement.select(
    "avg_active_power", 
    c("timestamp").dt.year().alias("year"),
    c("timestamp").dt.to_string(format="%m-%d %H:%M").alias("date_str"),
).sort("year").pivot(on="year", values="avg_active_power", index="date_str").sort("date_str")\
.with_columns(
    pl.coalesce("2021", "2024").alias("wind_data")
).select(
    (str(year) + "-" + c("date_str")).str.to_datetime(format="%Y-%m-%d %H:%M").alias("timestamp"),
    "wind_data"
)

fig = go.Figure()
fig.add_trace(
        go.Scatter(
            x=wind_production["timestamp"].to_list(), 
            y=wind_production["wind_data"].to_list(), 
            showlegend=False, line=dict(color=COLORS[0])
        )
    )

In [8]:

nb_days_list = list(turbined_volume.keys())

fig = make_subplots(
        rows=2, cols = 1, shared_xaxes=True, vertical_spacing=0.05, x_title="<b>Week<b>", 
        row_titles= [f"{nb_days_list[0]} days", f"{nb_days_list[1]} days"] )

for i, nb_days in enumerate(list(turbined_volume.keys())):
    fig.add_trace(
            go.Bar(
                x=turbined_volume[nb_days]["T"].to_list(), 
                y=(turbined_volume[nb_days]["turbined_volume"]/1e6).to_list(), 
                showlegend=False, marker=dict(color=COLORS[0]), width=1 
            ), row=i+1, col=1
    )

    fig.add_trace(
            go.Bar(
                x=pumped_volume[nb_days]["T"].to_list(), 
                y=(-pumped_volume[nb_days]["pumped_volume"]/1e6).to_list(), 
                showlegend=False, marker=dict(color="red"), width=1 
            ), row=i+1, col=1
    )


fig.update_layout(
    barmode='relative',
    margin=dict(t=60, l=65, r= 10, b=60), 
    width=1000,   # Set the width of the figure
    height=600,   #
    title= f"Turbined volume [Mm3]")

IndexError: list index out of range

In [ ]:
turbined_volume

{7: shape: (53, 2)
 ┌─────┬─────────────────┐
 │ T   ┆ turbined_volume │
 │ --- ┆ ---             │
 │ i64 ┆ f64             │
 ╞═════╪═════════════════╡
 │ 0   ┆ 1.4060e6        │
 │ 1   ┆ 1.4060e6        │
 │ 2   ┆ 1.4060e6        │
 │ 3   ┆ 1.4060e6        │
 │ 4   ┆ 1.4060e6        │
 │ …   ┆ …               │
 │ 48  ┆ 1.2455e6        │
 │ 49  ┆ 1.4060e6        │
 │ 50  ┆ 0.0             │
 │ 51  ┆ 0.0             │
 │ 52  ┆ 0.0             │
 └─────┴─────────────────┘,
 1: shape: (53, 2)
 ┌─────┬─────────────────┐
 │ T   ┆ turbined_volume │
 │ --- ┆ ---             │
 │ i64 ┆ f64             │
 ╞═════╪═════════════════╡
 │ 0   ┆ 1.4060e6        │
 │ 1   ┆ 1.4060e6        │
 │ 2   ┆ 1.4060e6        │
 │ 3   ┆ 1.4060e6        │
 │ 4   ┆ 1.4060e6        │
 │ …   ┆ …               │
 │ 48  ┆ 602588.16       │
 │ 49  ┆ 401725.44       │
 │ 50  ┆ 451642.86       │
 │ 51  ┆ 0.0             │
 │ 52  ┆ 0.0             │
 └─────┴─────────────────┘}